DATA ENGINEERING PLATFORMS (MSCA 31012)
Webscraping using Python ( Example 1 )
References: https://first-web-scraper.readthedocs.io/en/latest/ http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html http://altitudelabs.com/blog/web-scraping-with-python-and-beautiful-soup/

Installation: pip install BS4 | pip install Requests

In [11]:
!pip install BS4 
!pip install Requests
import warnings
warnings.filterwarnings("ignore")
import csv
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML

Scraping Rules
You should check a website’s Terms and Conditions before you scrape it. Be careful to read the statements about legal use of data. Usually, the data you scrape should not be used for commercial purposes.
Do not request data from the website too aggressively with your program (also known as spamming), as this may break the website. Make sure your program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.
The layout of a website may change from time to time, so make sure to revisit the site and rewrite your code as needed

In [12]:
# scrape the current Detainees of Boone County Jail from webpage into CSV
url = 'http://proximityone.com/ziptractequiv.htm#table'
headers = {'User-Agent': "Chrome/54.0.2840.90"}
response = requests.get(url, headers=headers)
html = response.content 

In [ ]:
#HTML('<iframe src=https://msdh.ms.gov/msdhsite/_static/14,0,420.html width=900 height=350><iframe>')

In [16]:
soup = BeautifulSoup(html, "lxml")  #Feed the HTML page to BeautifulSoup. 
table = soup.find('span', id="aw36-rows-start", attrs={'class': "aw-view-top"})

In [17]:
print(table)

None


In [15]:
tmpRow = (table.findAll('span')[1:])  
print (tmpRow)
#BeautifulSoup gets us going by allowing us to dig down into our table and return a list of 
#rows, which are created in HTML using <tr> tags inside the table.

AttributeError: 'NoneType' object has no attribute 'findAll'

In [ ]:
### Editing the code to fetch details of every record. The 'Case Number' and 'Charge Description' is Fetched for every Record.
list_of_rows = []
try:
    outfile = open("./PublicSchoolZip.csv", "w")
    writer = csv.writer(outfile)
    writer.writerow(["ZIPCode", "StAb", "StCtyTract", "CtyTract", "TractCode1", "TractCode2", "Population", "Housing Units"]) #instead of digging through headers, skip first row when and write headers here. 
    for row in table.findAll('span'):     #return list of rows which are created in HTML using <tr>
        list_of_cells = []
        for cell in row.findAll("span"):  #Loop through cells in each row. Cells are created in HTML by <td>
            #if(cell['align'] == ''):
            #    continue
            text = cell.text.replace('&nbsp;', '') #&nbsp; is html code for a non-breaking space which forces browser to render empty space. Delete it. 
            list_of_cells.append(text) #need CSV file (grid of columns and rows) do this by adding each cell in row to new Python list. 
        for anchor in row.findAll('th'):
            #href is fetched for every record and the response is parsed for every individual hit to href.
            details_scope = "http://proximityone.com/ziptractequiv.htm#table"+anchor['scope']
            details_response = requests.get(details_scope)
            details_html = details_response.content
            details_soup = BeautifulSoup(details_html, "html.parser")
            details_table = details_soup.find('table', attrs={'class': 'aw-templates-list aw-grid-view'})
            if details_table is not None:
                details_table_tr = details_table.find('span', attrs={"class": "aw-hpanel-bottom "})
                if details_table_tr is not None:
                    list_of_cells.append((details_table_tr.find('td', attrs = {'align' : 'CASE #'})).text) #lump lists into one big list of lists
                    list_of_cells.append((details_table_tr.find('td', attrs = {'alig' : 'CHARGE DESCRIPTION'})).text)
        arrLength = len(list_of_cells)
        writer.writerow(list_of_cells) #dump out list of lists. 
finally:
    outfile.close()  

In [ ]:
# data cleaning 
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('./PublicSchoolZip.csv')
df.head()

In [ ]:
# save final results
df.to_csv("./publicschoolZip.csv")